# 2.1 내장 시퀀스 개요

파이썬 표준 라이브러리는 C로 구현된 다음 시퀀스형을 제공한다.

* 컨테이너 시퀀스 (객체에 대한 참조를 담음)
    * 서로 다른 자료형의 항목들을 담을 수 있는 list, tuple, collections.deque 형
* 균일 시퀀스 (자신의 메모리 공간에 각 항목의 값을 담음 > 메모리를 더 적게 사용)
    * 단 하나의 자료형만 담을 수 있는 str, bytes, bytearray, memoryview, array.array 형
    
가변성에 따라 다음과 같이 분류할 수도 있다.

* 가변 시퀀스
    * list, bytearray, array.array, collections.deque, memoryview 형
* 불변 시퀀스
    * tuple, str, bytes 형

# 2.2 지능형 리스트와 제너레이터 표현식

지능형 리스트(리스트형의 경우)나 제너레이터 표현식(그 외 시퀀스의 경우)을 사용하면 가독성이 좋고 때로는 실행 속도도 빠른 코드를 만들 수 있다.

## 2.2.1 지능형 리스트와 가독성

다음 두 코드 중 어느 코드가 읽기 쉬운가?

In [1]:
symbols = "$*&!@"
codes = []
for symbol in symbols:
    codes.append(ord(symbol)) # ord : Return the Unicode code point for a one-character string.
print(codes)

[36, 42, 38, 33, 64]


In [2]:
symbols = "$*&!@"
codes = [ord(symbol) for symbol in symbols]
print(codes)

[36, 42, 38, 33, 64]


for 문은 다양한 일에 사용할 수 있다. 그러나 지능형 리스트는 오로지 새로운 리스트를 만드는 일만 한다. 

* 파이썬에서는 [], {}, () 안에서의 개행이 무시되므로 \ 를 사용하지 않고도 여러 줄에 걸쳐 리스트, 지능형 리스트, 제너레이터 표현식, 딕셔너리를 작성할 수 있다.

## 2.2.2 지능형 리스트와 map()/filter() 비교

지능형 리스트는 filter() 와 map() 함수를 이용해서도 구현할 수 있지만 가독성이 떨어진다.

In [5]:
symbols = "$*&!@"
test = [ord(s) for s in symbols if ord(s) > 40] # 지능형 리스트%
print(test)

[42, 64]


In [6]:
symbols = "$*&!@"
test = list(filter(lambda c: c>40, map(ord, symbols)))
print(test)

[42, 64]


In [10]:
map(ord, symbols), list(map(ord, symbols))

(<map at 0x7f817d76e6d8>, [36, 42, 38, 33, 64])

In [11]:
filter(lambda c: c>40, map(ord, symbols)), list(filter(lambda c: c>40, map(ord, symbols)))

(<filter at 0x7f817d76ee48>, [42, 64])

두 방법의 성능을 비교해보자.

In [17]:
import math

In [36]:
%%timeit
ans = [math.log(i) for i in range(1,100000000) if i>1000000]
print(len(ans))

98999999
98999999
98999999
98999999
98999999
98999999
98999999
98999999
12.4 s ± 121 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
ans = list(map(math.log, filter(lambda c: c>1000000, range(1,100000000))))
print(len(ans))

98999999
98999999
98999999
98999999
98999999
98999999
98999999
98999999
12.2 s ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


거의 차이가 없구만...

## 2.2.3 데카르트 곱

데카르트 곱은 두 개 이상의 리스트에 있는 모든 항목을 이용해서 만든 튜플로 구성된 리스트이며, 지능형 리스트를 통해서도 만들 수 있다.

In [39]:
colors = ['black', 'white']
sizes = list('SML')
tshirts = [(color, size) for size in sizes
                                  for color in colors]
print(tshirts)

[('black', 'S'), ('white', 'S'), ('black', 'M'), ('white', 'M'), ('black', 'L'), ('white', 'L')]


## 2.2.4 제너레이터 표현식

튜플, 배열 등의 시퀀스형을 초기화하려면 지능형 리스트를 사용할 수도 있지만, 다른 생성자에 전달할 리스트를 통째로 만들지 않고 반복자 프로토콜을 이용해서 항목을 하나씩 생성하는 제너레이터 표현식이 메모리를 더 적게 사용한다. 제너레이터 표현식은 대괄호 대신 괄호를 사용한다.

In [40]:
symbols = "$*&!@"
test = tuple(ord(s) for s in symbols if ord(s) > 40)
print(test)

(42, 64)


In [41]:
import array
test = array.array('I', (ord(symbol) for symbol in symbols))
print(test)

array('I', [36, 42, 38, 33, 64])


데카르트 곱을 만들기 위해 사용할 리스트에 각각 천 개의 항목이 들어있는 경우 제너레이터 표현식을 사용하면 단지 for 루프에 전달하기 위해 항목이 백만 개 들어 있는 리스트를 생성하는 일을 피할 수 있다.

In [54]:
colors = ['black', 'white']
sizes = list('SML')
for tshirt in ('{} {}'.format(color,size) for size in sizes for color in colors):
    print(tshirt)

black S
white S
black M
white M
black L
white L


# 2.3 튜플은 단순한 불변 리스트가 아니다.

튜플은 불변 리스트로 사용할 수도 있지만 필드명이 없는 레코드로 사용할 수도 있다.

## 2.3.1 레코드로서의 튜플

튜플의 각 항목은 레코드의 필드 하나를 의미하며 항목의 위치가 의미를 결정한다. 튜플을 단지 불변 리스트로 생각한다면 경우에 따라 항목의 크기와 순서가 중요하지 않을 수도 있다. 그러나 튜플을 필드의 집합으로 사용하는 경우에는 항목 수가 고정되어 있고 그 순서가 중요하다. 다음은 튜플을 레코드로 사용하는 예제이다.

In [56]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP','XDA205856')]

In [57]:
for passport in sorted(traveler_ids):
    print('%s/%s' % passport)

BRA/CE342567
ESP/XDA205856
USA/31195855


In [58]:
for country, _ in traveler_ids:
    print(country)

USA
BRA
ESP


## 2.3.2 튜플 언패킹

튜플의 각 항목을 새로운 변수에 할당하는 것을 튜플 언패킹이라고 한다. 위의 예제에서 `city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)`와 같은 부분을 이야기한다. 또한 다음과 같이 함수의 인수로 튜플을 입력할 때, 인수 앞에 `*`를 붙여 튜플을 언패킹할 수 있다.

In [59]:
divmod(20,8)

(2, 4)

In [63]:
t = (20,8)
divmod(t) # 튜플을 바로 인수로 입력하면 에러 발생

TypeError: divmod expected 2 arguments, got 1

In [61]:
divmod(*t)

(2, 4)

### 초과 항목을 잡기 위해 * 사용하기

In [66]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [67]:
a, b, *rest = range(3)
a, b, rest

(0, 1, [2])

In [68]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

In [69]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

In [70]:
a, *body, c = range(5)
a, body, c

(0, [1, 2, 3], 4)

In [71]:
*head, b, c, d = range(5)
head, b, c, d

([0, 1], 2, 3, 4)

## 2.3.4 명명된 튜플

`collections.namedtuple()` 함수는 필드명과 클래스명을 추가한 튜플의 서브클래스를 생성하는 함수로서, 디버깅할 때 유용하다. 필드명이 클래스에 저장되므로 `namedtuple()`로 생성한 객체는 튜플과 동일한 크기의 메모리만 사용한다.

In [72]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')

In [74]:
tokyo = City('Tokyo', 'JP', 36.933, (35.689, 139.691))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689, 139.691))

In [76]:
tokyo.population, tokyo[1]

(36.933, 'JP')

명명된 튜플은 튜플에서 상속받은 속성 외에 `_fields` 클래스 속성, `_make(iterable)` 클래스 메서드, `_asdict()` 객체 메서드 등을 가지고 있다.

In [77]:
City._fields

('name', 'country', 'population', 'coordinates')

In [78]:
LatLong = namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613, 77.208))

In [81]:
delhi = City._make(delhi_data) # City(*delhi_data) 와 동일한 기능
delhi

City(name='Delhi NCR', country='IN', population=21.935, coordinates=LatLong(lat=28.613, long=77.208))

In [85]:
delhi._asdict() # OrderedDict 객체로 변환

OrderedDict([('name', 'Delhi NCR'),
             ('country', 'IN'),
             ('population', 21.935),
             ('coordinates', LatLong(lat=28.613, long=77.208))])

In [86]:
for key, value in delhi._asdict().items():
    print(key + ':', value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: LatLong(lat=28.613, long=77.208)


## 2.3.5 불변 리스트로서의 튜플

리스트랑 튜플을 지원하는 메서드 리스트 확인

# 2.4 슬라이싱